# 06 - Signals & Hooks

## Use Case: Audit Logging & Real-Time Notifications

Signals let you hook into model lifecycle events (save, update, delete) without
modifying the model itself. This is the **observer pattern** applied to your ORM.

Common use cases:
- **Audit logging** — Record every change to a model for compliance
- **Real-time notifications** — Push WebSocket events after database writes
- **Cache invalidation** — Clear caches when underlying data changes
- **Metrics / timing** — Measure how long operations take

This notebook covers:

1. **Pre/Post Save** — Before and after a record is created or updated
2. **Pre/Post Delete** — Before and after a record is deleted
3. **Pre/Post Update** — Before and after a `merge()` / partial update
4. **Around Signals** — Generator-based middleware wrapping the entire operation
5. **Disconnecting Signals** — Cleanup to avoid side effects

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Python packages installed (`uv sync` in the project root)

In [ ]:
# Setup: add project root to path and configure the connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## Model Definitions

We define an **Article** model as the main entity and an **AuditLog** model to
record changes. Signals will automatically create audit entries.

In [ ]:
# Define the Article and AuditLog models
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class Article(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="articles")

    id: str | None = None
    title: str
    body: str = ""
    published: bool = False


class AuditLog(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="audit_logs")

    id: str | None = None
    table_name: str
    record_id: str
    action: str  # created, updated, deleted
    details: str = ""


print("Models defined: Article, AuditLog")

## 1. Pre/Post Save Signals

`pre_save` fires **before** the database write. Use it for validation or data transformation.

`post_save` fires **after** the database write. The `created` argument tells you whether
this was a new record or an update to an existing one.

Use cases:
- `pre_save`: Normalize data (e.g., strip whitespace, lowercase emails)
- `post_save`: Create audit log entries, send notifications, invalidate caches

In [ ]:
# Register pre_save and post_save signal handlers
from src.surreal_orm import pre_save, post_save

# Collect signal events in a list so we can inspect them
signal_log = []


@pre_save.connect(Article)
async def on_article_pre_save(sender, instance, **kwargs):
    """Normalize the title before saving."""
    instance.title = instance.title.strip()
    signal_log.append(f"pre_save: title normalized to '{instance.title}'")


@post_save.connect(Article)
async def on_article_post_save(sender, instance, created, **kwargs):
    """Log the save action after the database write."""
    action = "created" if created else "updated"
    signal_log.append(f"post_save: Article {action} — '{instance.title}'")

    # Create an audit log entry
    audit = AuditLog(
        table_name="articles",
        record_id=str(instance.id),
        action=action,
        details=f"Title: {instance.title}",
    )
    await audit.save()


print("Signal handlers registered: pre_save, post_save")

In [ ]:
# Create an article — triggers pre_save (normalize) and post_save (audit log)
article = Article(title="  Introduction to Signals  ", body="Signals are powerful hooks...")
await article.save()

print("Signal events:")
for event in signal_log:
    print(f"  {event}")

# Verify the audit log was created
audits = await AuditLog.objects().filter(table_name="articles").exec()
print(f"\nAudit log entries: {len(audits)}")
for a in audits:
    print(f"  [{a.action}] {a.details}")

## 2. Pre/Post Delete Signals

`pre_delete` fires **before** the record is removed. Use it to check constraints
or archive data before deletion.

`post_delete` fires **after** the record is removed. Use it for cleanup, cache
invalidation, or notifications.

In [ ]:
# Register delete signal handlers
from src.surreal_orm import pre_delete, post_delete


@pre_delete.connect(Article)
async def on_article_pre_delete(sender, instance, **kwargs):
    """Log before deletion happens."""
    signal_log.append(f"pre_delete: About to delete '{instance.title}'")


@post_delete.connect(Article)
async def on_article_post_delete(sender, instance, **kwargs):
    """Create audit entry after deletion."""
    signal_log.append(f"post_delete: Article deleted — '{instance.title}'")

    audit = AuditLog(
        table_name="articles",
        record_id=str(instance.id),
        action="deleted",
        details=f"Deleted: {instance.title}",
    )
    await audit.save()


print("Delete signal handlers registered")

In [ ]:
# Delete the article — triggers pre_delete and post_delete
signal_log.clear()
await article.delete()

print("Signal events during delete:")
for event in signal_log:
    print(f"  {event}")

# Verify audit logs now include the deletion
audits = await AuditLog.objects().filter(table_name="articles").exec()
print(f"\nTotal audit entries: {len(audits)}")
for a in audits:
    print(f"  [{a.action}] {a.details}")

## 3. Pre/Post Update Signals

`pre_update` and `post_update` fire around `merge()` calls (partial updates).
The `update_fields` dict tells you exactly which fields are being changed.

This is different from `save()` signals, which fire on full upserts.

In [ ]:
# Register update signal handlers
from src.surreal_orm import pre_update, post_update


@pre_update.connect(Article)
async def on_article_pre_update(sender, instance, update_fields, **kwargs):
    """Log which fields are about to change."""
    signal_log.append(f"pre_update: Fields changing: {list(update_fields.keys())}")


@post_update.connect(Article)
async def on_article_post_update(sender, instance, update_fields, **kwargs):
    """Create audit entry with field-level detail."""
    signal_log.append(f"post_update: Updated {list(update_fields.keys())}")

    audit = AuditLog(
        table_name="articles",
        record_id=str(instance.id),
        action="updated",
        details=f"Fields: {list(update_fields.keys())}",
    )
    await audit.save()


print("Update signal handlers registered")

In [ ]:
# Create a new article and then update it via merge()
signal_log.clear()

article2 = Article(title="Signals Deep Dive", body="Let's explore update signals.")
await article2.save()
print(f"Article created: '{article2.title}' (published={article2.published})")

# Partial update via merge() — triggers pre_update and post_update
signal_log.clear()
await article2.merge(published=True)

print("\nSignal events during merge():")
for event in signal_log:
    print(f"  {event}")

## 4. Around Signals — Generator-Based Middleware

Around signals wrap the **entire** operation using a Python generator. The `yield`
statement is where the database operation happens. This gives you:

- **Shared state** between before and after logic (local variables)
- **Guaranteed cleanup** with `try/finally` (even if the operation fails)
- **Timing** without needing separate pre/post handlers

Execution order: `pre_* -> around(before yield) -> DB operation -> around(after yield) -> post_*`

In [ ]:
# Register an around_save signal for timing and guaranteed cleanup
from src.surreal_orm import around_save, around_delete
import time


@around_save.connect(Article)
async def time_article_save(sender, instance, **kwargs):
    """Measure how long the save operation takes."""
    start = time.time()
    signal_log.append(f"around_save: Starting save for '{instance.title}'")

    yield  # <-- The actual save() happens here

    duration = time.time() - start
    signal_log.append(f"around_save: Save completed in {duration:.4f}s")


@around_delete.connect(Article)
async def guarded_delete(sender, instance, **kwargs):
    """Demonstrate guaranteed cleanup with try/finally."""
    resource = f"lock:article:{instance.id}"
    signal_log.append(f"around_delete: Acquired {resource}")
    try:
        yield  # <-- The actual delete() happens here
    finally:
        # This ALWAYS runs, even if delete() raises an exception
        signal_log.append(f"around_delete: Released {resource}")


print("Around signal handlers registered")

In [ ]:
# Demonstrate around_save: create an article and observe the timing
signal_log.clear()

article3 = Article(title="Around Signals in Action", body="This save is being timed.")
await article3.save()

print("Signal events (around_save):")
for event in signal_log:
    print(f"  {event}")

In [ ]:
# Demonstrate around_delete: guaranteed cleanup even on success
signal_log.clear()

await article3.delete()

print("Signal events (around_delete):")
for event in signal_log:
    print(f"  {event}")

## 5. Disconnecting Signals

Signals persist for the lifetime of the process. In notebooks, tests, or
temporary contexts, you should **disconnect** signals when done to avoid
unintended side effects.

- `signal.disconnect(handler, ModelClass)` — Remove a specific handler
- `signal.disconnect_all(ModelClass)` — Remove all handlers for a model

In [ ]:
# Disconnect a specific handler
pre_save.disconnect(on_article_pre_save, Article)
print("Disconnected: on_article_pre_save")

# Disconnect all handlers for a signal + model combination
post_save.disconnect_all(Article)
print("Disconnected: all post_save handlers for Article")

# Disconnect remaining signal handlers to clean up fully
pre_delete.disconnect_all(Article)
post_delete.disconnect_all(Article)
pre_update.disconnect_all(Article)
post_update.disconnect_all(Article)
around_save.disconnect_all(Article)
around_delete.disconnect_all(Article)

print("All Article signal handlers disconnected.")

In [ ]:
# Verify signals are disconnected: saving should produce no signal events
signal_log.clear()

test_article = Article(title="No Signals Here", body="This should be silent.")
await test_article.save()
await test_article.delete()

print(f"Signal events after disconnect: {len(signal_log)} (expected 0)")

## Cleanup

Remove all data created during this notebook.

In [ ]:
# Clean up all tables used in this notebook
await Article.raw_query("DELETE FROM articles")
await AuditLog.raw_query("DELETE FROM audit_logs")

print("Cleanup complete: all articles and audit logs deleted.")

## Summary

| Signal Type | When It Fires | Key Argument | Use Case |
|-------------|--------------|--------------|----------|
| `pre_save` | Before `save()` | `instance` | Validation, normalization |
| `post_save` | After `save()` | `instance`, `created` | Audit logging, notifications |
| `pre_delete` | Before `delete()` | `instance` | Constraint checks, archival |
| `post_delete` | After `delete()` | `instance` | Cleanup, cache invalidation |
| `pre_update` | Before `merge()` | `instance`, `update_fields` | Validation of changes |
| `post_update` | After `merge()` | `instance`, `update_fields` | Field-level audit logging |
| `around_save` | Wraps `save()` | `instance` (generator) | Timing, guaranteed cleanup |
| `around_delete` | Wraps `delete()` | `instance` (generator) | Locking, resource management |

**Important**: Always disconnect signals in notebooks and tests to prevent leaking handlers
into other code.